In [235]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [236]:
train_limpio = pd.read_csv("train_limpio_con_BOW_de_5000_y_Stemming_noDrops.csv",encoding = "ISO-8859-1")
test_limpio = pd.read_csv("test_limpio_con_BOW_de_5000_y_Stemming_noDrops.csv",encoding = "ISO-8859-1")

In [237]:
train_limpio["text"] = train_limpio["text"].map(lambda x : "null" if(x is None) else x)
test_limpio["text"] = test_limpio["text"].map(lambda x : "null" if(x is None) else x)

In [238]:
train_limpio_true = train_limpio.loc[train_limpio["target"] == 1].sample(frac=1).reset_index(drop=True)
train_limpio_false = train_limpio.loc[train_limpio["target"] == 0].sample(frac=1).reset_index(drop=True)

In [239]:
print("Tenemos {cant_verdaderos: .2f} verdaderos y {cant_falsos: .2f} falsos".format(cant_verdaderos=len(train_limpio_true), cant_falsos=len(train_limpio_false)))

Tenemos  3271.00 verdaderos y  4342.00 falsos


## Voy a balancear el train y el validation, misma cantidad de falsos y verdaderos en ambos

In [240]:
train = pd.concat([train_limpio_true.iloc[:2616], train_limpio_false.iloc[:3473]])
val = pd.concat([train_limpio_true.iloc[2616:3000], train_limpio_false.iloc[3473:3850]])
test = pd.concat([train_limpio_true.iloc[3000:], train_limpio_false.iloc[3850:]])

In [241]:
train.head(2)

,aa,ab,aba,abandon,abbott,abbswinston,abc,abcnew,abe,abil,...,wound-keyword,wreck-keyword,wreckag-keyword,isRealPlace,longitud_tweet,letras_seguidas,sentimiento,objetividad,keyword,location
0,0,0,0,0,0,0,0,0,0,0,...,False,False,False,True,90,2,0.118182,0.427273,cyclon,Philippines
1,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,110,1,0.500000,0.500000,trauma,NaN


In [242]:
vectorizer = CountVectorizer()
vectorizer.fit(train_limpio["text"].values.astype('U'))

X_train = vectorizer.transform(train["text"].values.astype('U'))
X_val  = vectorizer.transform(val["text"].values.astype('U'))
X_test = vectorizer.transform(test["text"].values.astype('U'))

actual_tp_test_only_text = vectorizer.transform(test_limpio["text"].values.astype('U'))

In [243]:
def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

## Primer modelo simple

In [244]:
input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['acc'])
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 10)                133800    
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 11        
Total params: 133,811
Trainable params: 133,811
Non-trainable params: 0
_________________________________________________________________


In [245]:
first_history = model.fit(X_train, train["target"].to_numpy(),
                    epochs=100,
                    verbose=True,
                    validation_data=(X_val, val["target"].to_numpy()),
                    batch_size=10)

Epoch 1/100
609/609 [==============================] - 1s 1ms/step - loss: 0.5389 - accuracy: 0.7674 - val_loss: 0.4723 - val_accuracy: 0.7924
Epoch 2/100
609/609 [==============================] - 1s 1ms/step - loss: 0.3385 - accuracy: 0.8706 - val_loss: 0.4614 - val_accuracy: 0.8029
Epoch 3/100
609/609 [==============================] - 1s 1ms/step - loss: 0.2342 - accuracy: 0.9157 - val_loss: 0.4795 - val_accuracy: 0.7937
Epoch 4/100
609/609 [==============================] - 1s 1ms/step - loss: 0.1698 - accuracy: 0.9414 - val_loss: 0.5282 - val_accuracy: 0.7871
Epoch 5/100
609/609 [==============================] - 1s 1ms/step - loss: 0.1284 - accuracy: 0.9565 - val_loss: 0.5745 - val_accuracy: 0.7714
Epoch 6/100
609/609 [==============================] - 1s 1ms/step - loss: 0.1029 - accuracy: 0.9639 - val_loss: 0.6309 - val_accuracy: 0.7700
Epoch 7/100
609/609 [==============================] - 1s 1ms/step - loss: 0.0861 - accuracy: 0.9675 - val_loss: 0.6754 - val_accuracy: 0.7661

Epoch 58/100
609/609 [==============================] - 1s 1ms/step - loss: 0.0346 - accuracy: 0.9828 - val_loss: 1.8425 - val_accuracy: 0.7332
Epoch 59/100
609/609 [==============================] - 1s 1ms/step - loss: 0.0339 - accuracy: 0.9814 - val_loss: 1.8462 - val_accuracy: 0.7332
Epoch 60/100
609/609 [==============================] - 1s 1ms/step - loss: 0.0340 - accuracy: 0.9823 - val_loss: 1.8567 - val_accuracy: 0.7293
Epoch 61/100
609/609 [==============================] - 1s 1ms/step - loss: 0.0333 - accuracy: 0.9826 - val_loss: 1.8582 - val_accuracy: 0.7267
Epoch 62/100
609/609 [==============================] - 1s 1ms/step - loss: 0.0333 - accuracy: 0.9824 - val_loss: 1.8632 - val_accuracy: 0.7293
Epoch 63/100
609/609 [==============================] - 1s 1ms/step - loss: 0.0334 - accuracy: 0.9821 - val_loss: 1.8652 - val_accuracy: 0.7280
Epoch 64/100
609/609 [==============================] - 1s 1ms/step - loss: 0.0328 - accuracy: 0.9826 - val_loss: 1.8768 - val_accuracy:

In [246]:
plot_history(first_history)

KeyError: 'acc'

In [250]:
loss, accuracy = model.evaluate(X_test, test["target"].to_numpy(), verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Testing Accuracy:  0.7510
Testing Accuracy:  0.9860


In [247]:
firstPredictions = list(map(lambda x: x[0], model.predict_classes(actual_tp_test_only_text)))

In [ ]:
firstPredictionsDF = pd.DataFrame(data={'id': test_limpio["id"], 'target': firstPredictions})
firstPredictionsDF.head(10)

In [ ]:
firstPredictionsDF.to_csv('NN_predictions_first_model.csv', index=False)

## Segundo modelo, agrego ciudades y Tokenizer en vez de count para las words

In [258]:
non_null_locations = train_limpio[train_limpio.location.notnull()]["location"].to_numpy()
non_null_locations

array(['Birmingham', 'Est. September 2012 - Bristol', 'AFRICA', ...,
       'Vancouver', 'UK', 'Lincoln'], dtype=object)

In [261]:
encoder = LabelEncoder()
locations = encoder.fit_transform(non_null_locations)
len(locations)

5080

In [262]:
encoder = OneHotEncoder(sparse=False)
locations = locations.reshape((5080, 1))
encoder.fit_transform(locations)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [263]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_limpio["text"].values.astype('U'))

X_train = tokenizer.texts_to_sequences(train["text"].values.astype('U'))
X_val = tokenizer.texts_to_sequences(val["text"].values.astype('U'))
X_test = tokenizer.texts_to_sequences(test["text"].values.astype('U'))

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print(train_limpio["text"].values[2])
print(X_train[2])

all resid ask to shelter in place are be notifi by offic no other evacu or shelter in place order are expect
[404, 671, 228, 27, 20, 325, 210, 248, 6, 1, 93, 5, 212, 67, 73]


In [267]:
maxlen = 280

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_val = pad_sequences(X_val, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

## Pruebo sin ciudades, solo texto, modelo max complejo con maxPooling y mejor embedding

In [265]:
embedding_dim = 50

model1 = Sequential()
model1.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model1.add(layers.GlobalMaxPool1D())
model1.add(layers.Dense(10, activation='relu'))
model1.add(layers.Dense(1, activation='sigmoid'))
model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])
model1.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 280, 50)           675500    
_________________________________________________________________
global_max_pooling1d (Global (None, 50)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                510       
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 676,021
Trainable params: 676,021
Non-trainable params: 0
_________________________________________________________________


In [269]:
second_history = model1.fit(X_train, train["target"].to_numpy(),
                    epochs=50,
                    verbose=False,
                    validation_data=(X_val, val["target"].to_numpy()),
                    batch_size=10)

loss, accuracy = model1.evaluate(X_test, test["target"].to_numpy(), verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))
plot_history(history)

Testing Accuracy:  0.7588


KeyError: 'acc'